In [67]:
# Import modules
import json

In [3]:
# Load data
train_js = json.load(open("../data/train.json", "r"))

In [4]:
# Get 4 examples of question / steps pairs
import random

random.seed(10)

random_sample = random.sample(train_js, 4)

In [57]:
import pandas as pd

def process_table(table):
    df = pd.read_html(table)[0]
    # display(df)
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    
    return df

process_table(random_sample[1]["annotation"]["amt_table"])

/var/folders/6b/lbjm_qyx51n9qc12vknzp4f00000gn/T/ipykernel_51704/2583410515.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]


,1,NaN,net deferred financing costs ( in $ millions )
0,2,as of december 31 2011,28
1,3,financing costs deferred ( 1 ),8
2,4,accelerated amortization due to refinancing ac...,-1 ( 1 )
3,5,amortization,-5 ( 5 )
4,6,as of december 31 2012,30
5,7,financing costs deferred ( 3 ),2
6,8,accelerated amortization due to refinancing ac...,2014
7,9,amortization,-5 ( 5 )
8,10,as of december 31 2013,27
9,11,financing costs deferred ( 4 ),10


In [69]:
# Construct question step pairs
question_step_pairs = [
    {"user_question" : r["qa"]["question"], 
        "followup_questions": "\n".join(r["annotation"]["dialogue_break"]), 
        "table": r["annotation"]["amt_table"], 
        "calculation_steps": r["annotation"]["step_list"],
        "calculation": r["qa"]["program_re"]}
    if r.get("qa") is not None
    else {"user_questions" : [r["qa_0"]["question"], r["qa_1"]["question"]], 
        "followup_questions": "\n".join(r["annotation"]["dialogue_break"]), 
        "table": r["annotation"]["amt_table"], 
        "calculation_steps": r["annotation"]["step_list"],
        "calculation": [r["qa_0"]["program_re"], r["qa_1"]["program_re"]]}
    for r in random_sample
]

In [70]:
question_step_pairs[0]

{'user_question': 'what is the growth rate in net revenue from 2010 to 2011?',
 'followup_questions': 'what was the change in the net revenue from 2010 to 2011?\nand how much does this change represent in relation to the net revenue in 2010?',
 'table': "<table class='wikitable'><tr><td>1</td><td></td><td>amount ( in millions )</td></tr><tr><td>2</td><td>2010 net revenue</td><td>$ 1043.7</td></tr><tr><td>3</td><td>mark-to-market tax settlement sharing</td><td>-195.9 ( 195.9 )</td></tr><tr><td>4</td><td>retail electric price</td><td>32.5</td></tr><tr><td>5</td><td>volume/weather</td><td>11.6</td></tr><tr><td>6</td><td>other</td><td>-5.7 ( 5.7 )</td></tr><tr><td>7</td><td>2011 net revenue</td><td>$ 886.2</td></tr></table>",
 'calculation_steps': ['subtract(886.2, 1043.7)', 'divide(A0, 1043.7)'],
 'calculation': 'divide(subtract(886.2, 1043.7), 1043.7)'}

In [71]:
question_step_pairs[1]

{'user_question': 'assuming the revolver is undrawn , what would the annual fee for the revolver be?',
 'followup_questions': 'what was the quarterly commitment fee on the unused portion of the revolving credit facility?\nand converted to the millions?',
 'table': "<table class='wikitable'><tr><td>1</td><td></td><td>net deferred financing costs ( in $ millions )</td></tr><tr><td>2</td><td>as of december 31 2011</td><td>28</td></tr><tr><td>3</td><td>financing costs deferred ( 1 )</td><td>8</td></tr><tr><td>4</td><td>accelerated amortization due to refinancing activity ( 2 )</td><td>-1 ( 1 )</td></tr><tr><td>5</td><td>amortization</td><td>-5 ( 5 )</td></tr><tr><td>6</td><td>as of december 31 2012</td><td>30</td></tr><tr><td>7</td><td>financing costs deferred ( 3 )</td><td>2</td></tr><tr><td>8</td><td>accelerated amortization due to refinancing activity</td><td>2014</td></tr><tr><td>9</td><td>amortization</td><td>-5 ( 5 )</td></tr><tr><td>10</td><td>as of december 31 2013</td><td>27</td><

In [73]:
question_step_pairs[2]

{'user_questions': ['what was the percentage change in share-based compensation expense between 2011 and 2012?',
  'what was the percentage change in share-based compensation expense between 2012 and 2013?'],
 'followup_questions': 'what was the change in the share-based compensation expense from 2011 to 2012?\nand what was this change as a percent of that expense in 2011?\nand throughout the subsequent year of this period, what was that change?\nwhat percentage did this 2012-2013 change represent in relation to that compensation expense in 2012?',
 'table': "<table class='wikitable'><tr><td>1</td><td>for the years ended december 31,</td><td>2013</td><td>2012</td><td>2011</td></tr><tr><td>2</td><td>stock options</td><td>$ 24.7</td><td>$ 32.4</td><td>$ 41.7</td></tr><tr><td>3</td><td>rsus and other</td><td>23.8</td><td>22.6</td><td>18.8</td></tr><tr><td>4</td><td>total expense pre-tax</td><td>48.5</td><td>55.0</td><td>60.5</td></tr><tr><td>5</td><td>tax benefit related to awards</td><td

In [74]:
question_step_pairs[3]

{'user_question': 'what was the three year average interest rate for 2007-2009?',
 'followup_questions': 'combined, what was the risk-free interest rate in 2008 and 2009?\nand in 2007?\nwhat was the total for all three years?\nso what was the average value during this time?',
 'table': "<table class='wikitable'><tr><td>1</td><td></td><td>2009</td><td>2008</td><td>2007</td></tr><tr><td>2</td><td>dividend yield</td><td>0.0% ( 0.0 % )</td><td>1.9% ( 1.9 % )</td><td>1.3% ( 1.3 % )</td></tr><tr><td>3</td><td>expected stock price volatility</td><td>55.0% ( 55.0 % )</td><td>31.4% ( 31.4 % )</td><td>28.0% ( 28.0 % )</td></tr><tr><td>4</td><td>risk-free interest rate</td><td>1.8% ( 1.8 % )</td><td>2.8% ( 2.8 % )</td><td>4.8% ( 4.8 % )</td></tr><tr><td>5</td><td>expected option life</td><td>5 years</td><td>5 years</td><td>5 years</td></tr></table>",
 'calculation_steps': ['add(1.8, 2.8)',
  'Ask for number 4.8',
  'add(A0, 4.8)',
  'divide(A1, const_3)'],
 'calculation': 'divide(add(add(1.8, 2.8

In [76]:
pd.read_html(question_step_pairs[3]["table"])[0]

/var/folders/6b/lbjm_qyx51n9qc12vknzp4f00000gn/T/ipykernel_51704/741810508.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(question_step_pairs[3]["table"])[0]


,0,1,2,3,4
0,1,NaN,2009,2008,2007
1,2,dividend yield,0.0% ( 0.0 % ),1.9% ( 1.9 % ),1.3% ( 1.3 % )
2,3,expected stock price volatility,55.0% ( 55.0 % ),31.4% ( 31.4 % ),28.0% ( 28.0 % )
3,4,risk-free interest rate,1.8% ( 1.8 % ),2.8% ( 2.8 % ),4.8% ( 4.8 % )
4,5,expected option life,5 years,5 years,5 years
